In [1]:
%matplotlib inline
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/opt/conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
df = pd.read_csv('../data/data_only_keras.csv', names = ['x_acc', 'y_acc', 'z_acc'])
label_df = pd.read_csv('../data/labels_only_keras.csv', header = None)
print(df.head())
print(label_df.head())

      x_acc     y_acc     z_acc
0  0.600076  0.283347  0.649533
1  0.602326  0.283343  0.648932
2  0.599470  0.283345  0.648904
3  0.602812  0.283335  0.647378
4  0.602726  0.283329  0.645819
   0   1   2   3   4   5   6   7   8   9   10  11
0   1   0   0   0   0   0   0   0   0   0   0   0
1   1   0   0   0   0   0   0   0   0   0   0   0
2   1   0   0   0   0   0   0   0   0   0   0   0
3   1   0   0   0   0   0   0   0   0   0   0   0
4   1   0   0   0   0   0   0   0   0   0   0   0


In [3]:
traindf = df[ : int(0.7 * len(df))]
testdf = df[int(0.85 * len(df)) : ]
valdf = df[int(0.7 * len(df)) : int(0.85 * len(df))]
trainlabel = label_df[ : int(0.7 * len(df))]
testlabel = label_df[int(0.85 * len(df)) : ]
vallabel = label_df[int(0.7 * len(df)) : int(0.85 * len(df))]

train_gen = TimeseriesGenerator(traindf.values, trainlabel.values, length = 100, batch_size = 16, stride = 100)
test_gen = TimeseriesGenerator(testdf.values, testlabel.values, length = 100, batch_size = 1, stride = 100)
val_gen = TimeseriesGenerator(valdf.values, vallabel.values, length = 100, batch_size = 1, stride = 100)

In [6]:
n_input = 100
n_features = 3
batch_size = 1
model = Sequential()
model.add(keras.layers.GRU(80, activation = 'relu', input_shape = (n_input, n_features), return_sequences = True))
model.add(keras.layers.GRU(60, activation = 'relu', return_sequences = True))
model.add(keras.layers.GRU(40, activation = 'relu'))
model.add(Dense(12, activation = 'softmax'))
adam = keras.optimizers.Adam(lr = 5e-4)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit_generator(train_gen, validation_data = val_gen, epochs = 50, verbose = 1)

In [ ]:
model.evaluate_generator(test_gen)

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()